# Proyecto 2 Reconocimiento de Patrones

José Julián Camacho Hernández

Leonardo Guillén Fernández

In [131]:
import os
import time
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from IPython.display import display
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, f1_score

# MLP

In [155]:
def computeMetrics(y_test, y_pred, training_time):
    acc = accuracy_score(y_test, y_pred)                        # Calcular la exactitud
    f1 = f1_score(y_test, y_pred, average='macro')              # Calcular F1 score
    rec = recall_score(y_test, y_pred, average='macro')         # Calcular el recall
    prec = precision_score(y_test, y_pred, average='macro')     # Calcular la precisión
    #auc = roc_auc_score(y_test, y_pred, average='macro')        # Calcular AUC , "AUC":auc
    metrics = {"Accuracy":acc, "Precision":prec, "Recall":rec, "F1 Score":f1, "Tiempo de entrenamiento [s]":training_time}
    df = pd.DataFrame(metrics, index = [0])
    display(df)

In [32]:
def lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_image = np.zeros_like(gray)
    for i in range(1, gray.shape[0] - 1):
        for j in range(1, gray.shape[1] - 1):
            binary = ""
            center = gray[i, j]
            for x, y in [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]:
                if gray[i + x, j + y] >= center:
                    binary += "1"
                else:
                    binary += "0"
            lbp_image[i, j] = int(binary, 2)
    return lbp_image.flatten()

In [92]:
lbp_arrays = []
labels = []

In [94]:
input_folder = "./testi/"
# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        # Apply LBP and store the output array
        lbp_array = lbp(image)
        lbp_arrays.append(lbp_array)
        labels.append(3)


input_folder = "./testi2/"
# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        # Apply LBP and store the output array
        lbp_array = lbp(image)
        lbp_arrays.append(lbp_array)
        labels.append(2)

    
input_folder = "./testi3/"
# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        # Apply LBP and store the output array
        lbp_array = lbp(image)
        lbp_arrays.append(lbp_array)
        labels.append(0)


input_folder = "./testi4/"
# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        # Apply LBP and store the output array
        lbp_array = lbp(image)
        lbp_arrays.append(lbp_array)
        labels.append(1)
        

In [95]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

class CustomDataset:
    def __init__(self, lbp_tensors, labels):
        self.lbp_tensors = lbp_tensors
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        lbp_tensor = self.lbp_tensors[index]
        label = self.labels[index]
        return lbp_tensor, label

In [156]:
#=====  Feature Engineering  =====#

# Convertir las listas a numpy array de tipo uint8
lbp_np_arrays = np.array(lbp_arrays, dtype=np.uint8)
labels_np = np.array(labels)

# Dividir los datos en train y test y estratificar
train_lbp, test_lbp, train_labels, test_labels = train_test_split(
    lbp_np_arrays, labels_np, test_size=0.2, stratify=labels, random_state=42)

# Estandarización de los datos
scaler = MinMaxScaler()
scaler.fit(train_lbp)
scaled_data = scaler.transform(train_lbp)
train_lbp = scaled_data

# Convert the numpy array to a PyTorch tensor
lbp_train_tensor = torch.from_numpy(train_lbp).float()
lbp_test_tensor = torch.from_numpy(test_lbp).float()
labels_train_tensor = torch.from_numpy(train_labels).float()
labels_test_tensor = torch.from_numpy(test_labels).float()


In [159]:
# Instaciar modelo
input_size = lbp_tensors.shape[1]
hidden_size = 64
output_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_size, hidden_size, output_size)

# Definir la función de loss y optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

start_time = time.time()
# Entrenar el modelo
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(lbp_train_tensor)

    labels_train_long_tensor = labels_train_tensor.long()         
    labels_train_long_tensor = labels_train_long_tensor.to(device)
    labels_onehot = F.one_hot(labels_train_long_tensor, num_classes=output_size).float()

    loss = criterion(outputs, labels_onehot)
    loss.backward()
    optimizer.step()
end_time = time.time()
time_taken = end_time - start_time

# Evaluate the model
outputs = model(lbp_test_tensor)
_, predicted = torch.max(outputs.data, 1)

# Obtener métricas
computeMetrics(predicted, labels_test_tensor, time_taken)

/home/jose/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,0.868421,0.75,0.646429,0.693732,7.424655
